In [1]:
from PIL import Image 
import glob
import os
import random
import json

In [1]:
def extract_box(annotation):
    # Returns a tuple of the box coordinates from an annotation
    # from Sloth
    
    box_xl = int(annotation["x"])
    box_yt = int(annotation["y"])
    box_xr = int(box_xl + annotation["width"])
    box_yb = int(box_yt + annotation["height"])
    
    return box_xl, box_yt, box_xr, box_yb

def make_crop_coordinates(box_coords, n, m):
    
    # This function "smart crops" the images
    # It attempts to make n crops that have a random
    # size between 256 and 720, and m crops that are of size 720.
    img_xl = 0
    img_yt = 0
    img_xr = 1080
    img_yb = 720
    
    box_xl = box_coords[0]
    box_yt = box_coords[1]
    box_xr = box_coords[2]
    box_yb = box_coords[3]
    
    crop_coords = []
    
    # Try to get n crops that have random size between 256 and 720
    for i in range(n):
        
        # Get random top left corner that is above and left
        # of the bounding box
        crop_xl = random.randrange(img_xl,box_xl)
        crop_yt = random.randrange(img_yt,box_yt)
        
        # This while loop attempts to get a valid
        # bottom right corner for the already created
        # top left
        valid = False
        cntr = 0
        
        while not valid and cntr < 20:
            # Get the proposed size
            size = random.randrange(256, 720)
            # Get the proposed coords
            p_crop_xr = crop_xl + size
            p_crop_yb = crop_yt + size
            # Check if the coords are valid
            if p_crop_xr <= img_xr and p_crop_xr >= box_xr:
                if p_crop_yb <= img_yb and p_crop_yb >= box_yb:
                    valid = True
            # Cntr will go up
            # if cntr gets to 20 w/ out being valid...
            cntr += 1             
        #... the loop will continue to the next iteration
        if not valid:
            continue
        # If it is valid
        crop_xr = p_crop_xr
        crop_yb = p_crop_yb
        
        # Add the crop coordinates to the list
        crop_coords.append((crop_xl, crop_yt, crop_xr, crop_yb))
        
    # Try to get m crops that are 720 x 720
    for i in range(m):
        valid = False
        cntr = 0
        
        while not valid and cntr < 20:
            p_crop_xl = random.randrange(img_xl,box_xl)
            if p_crop_xl + 720 <= img_xr and p_crop_xl + 720 >= box_xr:
                valid = True
            cntr +=1
            
        if not valid:
            continue
        crop_xl = p_crop_xl
        crop_yt = 0
        crop_xr = crop_xl + 720
        crop_yb = 720
        
        crop_coords.append((crop_xl, crop_yt, crop_xr, crop_yb))

    # Return the crops
    return crop_coords
    
    

In [304]:
# Set the path settings here
# which have to match the locations, etc.
# from Sloth

path = "photos_trees_original"
save_path = "trees_cropped_v2/"
annotation_file = path + "/annotations_trees.json"
annotation_path_additition = '../../Self_portrait/'

annotation_dic = {}

# Load the annotations into a dictionary,
# accesible by the file name key
with open(annotation_file) as json_file:
    annotations = json.load(json_file)
    for i in range(len(annotations)):
        annotation_dic[annotations[i]['filename']] = annotations[i]['annotations'] 

# Start the crop loop
j= 0
for fn in glob.glob( path + '/*.jpg'):
    new_ims = []
    # Load an image
    im = Image.open(fn)
    
    # Get the annotation
    annotation = annotation_dic[annotation_path_additition+fn][0]
    # Get the box
    box_coords = extract_box(annotation)
    # Try to make 25 crops
    crop_coords = make_crop_coordinates(box_coords, 20, 5)
    
    # Add the succesful crops
    for i in range(len(crop_coords)):
        new_ims.append( im.crop(crop_coords[i]) )
        
    # Save the newly cropped images
    for new_im in new_ims:
        new_im = new_im.save(save_path+str(j) + "_cropped.jpg") 
        j+=1